In [ ]:
import telegram
from telegram import ReplyKeyboardMarkup
from telegram.ext import Updater
from telegram.ext import CommandHandler, RegexHandler, MessageHandler, Filters, ConversationHandler
from datetime import datetime
from db import Event, connect_default
import logging

# Enable logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)

logger = logging.getLogger(__name__)

CHOOSING, TYPING_REPLY, TYPING_CHOICE = range(3)

event_fields = ['Название ивента', 'Описание', "Ссылка на ивент", "Ссылка на регистрацию", "Место",
                "Время", "Категория", 'Компания', "Закончил"]
event_fields_dict = {'Название ивента': 'name', 'Описание': 'description', "Ссылка на ивент": 'link',
                     "Ссылка на регистрацию": 'link_reg', "Место": 'place', "Время": 'time'}

profiles = ["Бизнес", "Карьера", "Технологии", "Искуство", "Chill"]
organizations = ["1", "2"]
dict_class_names = {"1": "chose_1", "2": 'chose_2', "Бизнес": 'th_business',
                    "Карьера": 'th_career', "Технологии": 'th_tech', "Искуство": 'th_arts', "Chill": 'th_chill'}

reply_keyboard = [event_fields]
markup = ReplyKeyboardMarkup(reply_keyboard, one_time_keyboard=True, resize_keyboard=True)


def string4regex(event_fields):
    event_regex_str = ''
    for i in event_fields[:-1]:
        event_regex_str = event_regex_str + '|' + i
    event_regex_str = '^(' + event_regex_str[1:] + ")$"
    return event_regex_str


def facts_to_str(user_data):
    facts = list()

    for key, value in user_data.items():
        facts.append('{} - {}'.format(key, value))

    return "\n".join(facts).join(['\n', '\n'])


def start(bot, update):
    text = 'Привет! Это секретный бот для оргов'
    keyboard = [["Добавить мероприятие", "Помощь"]]
    reply_markup = ReplyKeyboardMarkup(
        keyboard, one_time_keyboard=True, resize_keyboard=True)
    bot.send_message(chat_id=update.message.chat_id, text=text, parse_mode='Markdown',
                     reply_markup=reply_markup)


def add(bot, update):
    update.message.reply_text("Выбери и заполни категории, нужно заполнить все!", reply_markup=markup)

    return CHOOSING


def regular_choice(bot, update, user_data):
    text = update.message.text
    user_data['choice'] = text
    if text == 'Время':
        update.message.reply_text("Введи время в таком формате: день.месяц.год_двумя_цифрами (пробел) часы:минуты. "
                                  "Пример: '02.06.18 16:03'")
    if text == 'Компания':
        reply_markup = ReplyKeyboardMarkup(keyboard=[organizations], one_time_keyboard=True, resize_keyboard=True)
        update.message.reply_text('Выбери компанию, к которой относится твоё мероприятие', reply_markup=reply_markup)

    if text == 'Категория':
        reply_markup = ReplyKeyboardMarkup(keyboard=[profiles], one_time_keyboard=True, resize_keyboard=True)
        update.message.reply_text("Выбери профиль мероприятия", reply_markup=reply_markup)

    else:
        update.message.reply_text(
            'Ты выбрал "{}"? Напиши всё одним собщением, как это будут видеть читатели рассылки'.format(text.lower()))

    return TYPING_REPLY


def received_information(bot, update, user_data):
    text = update.message.text
    category = user_data['choice']
    user_data[category] = text
    if 'Время' in user_data:
        try:
            datetime.strptime(user_data['Время'], '%d.%m.%y %H:%M')
        except:
            update.message.reply_text('Неправильно заполнил время, попробуй еще раз)')
    del user_data['choice']

    update.message.reply_text("Отлично, вот что ты сказал:"
                              "{}"
                              "Ты можешь сказать еще или исправить старые записи".format(
        facts_to_str(user_data)), reply_markup=markup)

    return CHOOSING


meta, session, base, db = connect_default()


def done(bot, update, user_data):
    if 'choice' in user_data:
        del user_data['choice']

    try:
        event = Event()
        for key, value in event_fields_dict.items():
            setattr(event, value, user_data[key])
        for key, value in dict_class_names.items():
            if (key == user_data['Компания']) or (key == user_data['Категория']):
                setattr(event, value, True)
        setattr(event, 'is_confirmed', True)
        session.add(event)
        session.commit()

        update.message.reply_text("В итоге ты заполнил поля вот так:"
                                  "{}"
                                  "Тыкай /start чтобы начать снова :)".format(facts_to_str(user_data)))
        user_data.clear()
    except:
        update.message.reply_text('Что-то пошло не так, возможно заполнил не все поля. ')

    return ConversationHandler.END


def error(bot, update, error):
    """Log Errors caused by Updates."""
    logger.warning('Update "%s" caused error "%s"', update, error)


def main():
    # Create the Updater and pass it your bot's token.

    token = ''
    bot = telegram.Bot(token)
    updater = Updater(token=token)
    dp = updater.dispatcher

    # Add conversation handler with the states GENDER, PHOTO, LOCATION and BIO
    conv_handler = ConversationHandler(
        entry_points=[RegexHandler('^Добавить мероприятие$', add)],

        states={
            CHOOSING: [RegexHandler(string4regex(event_fields), regular_choice, pass_user_data=True)],

            # TYPING_CHOICE: [MessageHandler(Filters.text, regular_choice, pass_user_data=True)],

            TYPING_REPLY: [MessageHandler(Filters.text, received_information, pass_user_data=True)],
        },

        fallbacks=[RegexHandler('^Закончил$', done, pass_user_data=True)]
    )

    dp.add_handler(CommandHandler('start', start))
    dp.add_handler(conv_handler)
    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()


if __name__ == '__main__':
    main()
